## MOwNiT - laboratorium 10
### Singular Value Decomposition
http://home.agh.edu.pl/~czech/mownit-lab/mownit-lab6.pdf

In [1]:
import os
import re
import collections
import numpy as np

from heapq import nlargest
from urllib.request import urlopen
from bs4 import BeautifulSoup

1. Przygotuj duży (>1000 elementów) zbiór dokumentów tekstowych w języku angielskim (np. wybrany korpus tekstów, podzbiór artykułów Wikipedii, zbiór dokumentów HTML uzyskanych za pomocą
Web crawlera, zbiór rozdziałów wyciętych z różnych książek)

In [2]:
# szybki crawler
global PAGE 
PAGE = "http://www.stewartonbibleschool.org.uk/bible/text/"
url_table = []

html_page = urlopen(PAGE)
soup = BeautifulSoup(html_page)
for link in soup.findAll('a'):
    a = link.get('href')
    if ".txt" in a:
        url_table.append(a)
print(url_table)

['genesis.txt', 'exodus.txt', 'levit.txt', 'numbers.txt', 'deut.txt', 'joshua.txt', 'judges.txt', 'ruth.txt', '1samuel.txt', '2samuel.txt', '1kings.txt', '2kings.txt', '1chron.txt', '2chron.txt', 'ezra.txt', 'nehemiah.txt', 'esther.txt', 'job.txt', 'psalms.txt', 'proverbs.txt', 'eccl.txt', 'song.txt', 'isaiah.txt', 'jeremiah.txt', 'lament.txt', 'ezekiel.txt', 'daniel.txt', 'hosea.txt', 'joel.txt', 'amos.txt', 'obadiah.txt', 'jonah.txt', 'micah.txt', 'nahum.txt', 'habakkuk.txt', 'zeph.txt', 'haggai.txt', 'zech.txt', 'malachi.txt', 'matthew.txt', 'mark.txt', 'luke.txt', 'john.txt', 'acts.txt', 'romans.txt', '1corinth.txt', '2corinth.txt', 'galatian.txt', 'ephesian.txt', 'philipp.txt', 'colossia.txt', '1thess.txt', '2thess.txt', '1timothy.txt', '2timothy.txt', 'titus.txt', 'philemon.txt', 'hebrews.txt', 'james.txt', '1peter.txt', '2peter.txt', '1john.txt', '2john.txt', '3john.txt', 'jude.txt', 'rev.txt']


In [3]:
bible = []
chunk_size = 2000
option = 0 # 0 => podział na księgi => 66 dokumentów
           # 1 => podział każdej księgi na fragmenty po 2000 znaków => 2121 dokumentów

for url in url_table:
    book = urlopen(PAGE + url).read().decode("utf-8")
    book = re.sub(r"\d+:\d+:", "", book)
    
    if option == 1:
        bookparts = []
        for i in range(0, len(book), chunk_size):
            bookparts.append(book[i:i + chunk_size])
        bible += bookparts
    else:
        bible.append(book)
    
len(bible)

66

2. Określ  słownik  słów  kluczowych  (termów)  potrzebny  do  wyznaczenia  wektorów
cech _bag-of-words_ (indeksacja). 
Przykładowo zbiorem takim może być **unia** wszystkich słów występujących we wszystkich tekstach.

In [4]:
words_in_book = [collections.Counter(re.findall(r'\w+', book)) for book in bible]
bag_of_words = sum(words_in_book, collections.Counter())
len(bag_of_words)

13619

3. Dla każdego dokumentu $j$ wyznacz wektor cech _bag-of-words_
$d_j$ zawierający częstości występowania poszczególnych słów (termów) w tekście.
4. Zbuduj rzadką macierz wektorów cech _term-by-document matrix_
w której wektory cech ułożone są kolumnowo $A_{m \times n} = [d_1 | d_2 | ... | d_n]$ 
($m$ jest liczbą termów  w słowniku, a $n$ liczbą dokumentów)

In [5]:
m = len(bag_of_words)
n = len(bible)
A = np.zeros((n, m))
i = 0

for book in bible:
    words_in_book = [collections.Counter(re.findall(r'\w+', book))]
    vector = sum(words_in_book, collections.Counter())
    d = np.zeros(m)
    j = 0
    for word in set(bag_of_words):
        if vector[word] > 0:
            d[j] = vector[word]
        j += 1

    A[i] = d
    i += 1
 
A = A.transpose()
print(A.shape)

(13619, 66)


In [6]:
# set(bag_of_words)

5. Przetwórz wstępnie otrzymany zbiór danych mnożąc elementy
_bag-of-words_ przez _inverse document frequency_. 
Operacja ta pozwoli na redukcję znaczenia często występujących słów.

$ IDF(w) = \log\frac{N}{n_w} $, gdzie $n_w$ jest liczbą dokumentów, w których występuje słowo
$w$, a $N$ jest całkowitą liczbą dokumentów.


In [7]:
idf = np.zeros(m)

w = 0
for word in set(bag_of_words):
    nw = 0;
    row = A[w]
    for i in range(0, n):
        if row[i] > 0:
            nw += 1
    idf[w] = np.log(n / nw)
    w += 1

6. Napisz  program  pozwalający  na  wprowadzenie  zapytania  (w  postaci  sekwencji słów)  przekształcanego  następnie  do  reprezentacji  wektorowej
$q$ (_bag-of-words_). Program ma zwrócić $k$ dokumentów najbardziej zbliżonych do podanego zapytania $q$. Użyj korelacji między wektorami jako miary podobieństwa.

In [8]:
def rate_of_similarity(sentence, k):
    query = re.sub("[^\w]", " ",  sentence).lower().split()
    
    q = np.zeros(m)
    j = 0
    for word in query:
        if word in set(bag_of_words):
            q[j] = 1
        j += 1
    
    similarity_rate = {}
    for i in range(0, 66):
        dj = A[:,[i]]
        q_norm = np.linalg.norm(q)
        dj_norm = np.linalg.norm(dj)
        cosj = np.dot(q,dj) / (q_norm * dj_norm)
        similarity_rate.update({i:cosj})
        
    return nlargest(k, similarity_rate, key=similarity_rate.get)

In [9]:
sentence = "And a river went out of Eden to water the garden"
simple = rate_of_similarity(sentence, 15)

8. W  celu  usunięcia  szumu  z  macierzy $A$
zastosuj  SVD  i _low rank approximation_ otrzymując: [długi wzór]

In [10]:
U, s, V = np.linalg.svd(A, full_matrices = False)
S = np.diag(s)
np.allclose(A, np.dot(U, np.dot(S, V)))
A = np.dot(U, np.dot(S, V))

withSVD = rate_of_similarity(sentence, 15)

9. Porównaj działanie programu bez usuwania szumu i z usuwaniem szumu. 
Dla jakiej wartości $k$ wyniki wyszukiwania są najlepsze (subiektywnie). Zbadaj wpływ
przekształcenia IDF na wyniki wyszukiwania.

In [11]:
for i in range(0, m):
    for j in range(0, n):
        A[i][j] = A[i][j] * idf[i]
        
withIDF = rate_of_similarity(sentence, 15)

In [12]:
print(simple)
print(withSVD)
print(withIDF)

[9, 2, 7, 46, 43, 18, 27, 41, 8, 12, 11, 40, 19, 17, 39]
[9, 2, 7, 46, 43, 18, 27, 41, 8, 12, 11, 40, 19, 17, 39]
[2, 9, 46, 43, 41, 27, 18, 7, 40, 17, 19, 12, 39, 11, 23]


A dla option = 1, czyli 2121 dokumentów laptop prawie odlatuje, ale ostatecznie wyniki to:

simple = $\texttt{[1950, 18, 44, 1951, 31, 680, 1787, 49, 494, 226, 67, 388, 2000, 1949, 1720]}$

withSVD = $\texttt{[1950, 18, 44, 1951, 31, 680, 1787, 49, 494, 226, 67, 388, 2000, 1949, 1720]}$

withIDF = $\texttt{[1950, 1787, 71, 18, 1720, 680, 31, 1044, 44, 1075, 1224, 1951, 388, 49, 2000]}$

Wnioski: 
- zastosowanie odszumiania nie zmienia wyników,
- zastosowanie IDF zmienia kolejność wyników.